# Interactive webmap using python

Important packages to be used are:
    i. $GeoPandas$ to read and analyze shapefiles,
    ii. $Folium$ to make html maps,
    iii. $Pandas$ to handle tabular data,
    iv. $Numpy$ to perform numerical tasks, and
    v. $matplotlib$ to publish 2D figures.

In [ ]:
# loading packages
import os
import geopandas as gpd
import folium
import base64
from folium import IFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
print("Successful!")

### Load both attribute data and spatial data.

I'm using pandas to load attribute data and geopandas to load spatial data.

In [ ]:
# Attribute data.
att_data = pd.read_csv("cornYield.csv")
att_data.tail()

In [ ]:
# Spatial Data
spatialData = gpd.read_file(r"NC_USA/NorthCentral.shp")
spatialData.head(5)

In [ ]:
spatialData.shape

In [ ]:
spatialData.explore()

### Attribute Data Cleaning.

In [ ]:
# convert the year int column into datetime
att_data["Year"] = att_data["Year"].astype("str") + "-12-31"
att_data["Year"] = pd.to_datetime(att_data.Year, format = "%Y-%m-%d")
att_data.head()

In [ ]:
# Remove unnecessary columns
att_data = att_data.drop(columns=['County ANSI', 'Program', 'Period', 'Week Ending', 'Geo Level', 'State ANSI',
                                  'Ag District', 'Ag District Code', 'Zip Code', 'Region', 'watershed_code', 'Watershed',
                                 'Commodity', 'Data Item', 'Domain Category', 'Domain'])
att_data.head()

In [ ]:
null_count = att_data.isnull().sum()
null_count # Total of 841 columns are empty

In [ ]:
att_data['CV (%)'] = att_data['CV (%)'].fillna(0)
att_data.tail()

In [ ]:
null_count = att_data.isnull().sum()
null_count

In [ ]:
# creating a column that holds both county and state through concatenation.
att_data['State_County'] = att_data['State'] + '_' + att_data['County']
att_data.tail()

In [ ]:
null_count = att_data.isnull().sum()
null_count # our data has no null value

### Calculating trend slopes.

In [ ]:
def calculate_slopes(data):
    # Get unique state county combination
    stateCountyUnique = np.unique(att_data['State_County'].values)
    # empty dictionary to hold information
    slopes = {}
    #running a loop through each unique counties and calculating slope
    for uniq in stateCountyUnique:
        # sort by year
        yield_val = data[att_data['State_County'] == uniq].sort_values(by='Year')
        # Set the year as new index.
        yield_val = yield_val.set_index('Year')
        slope, inter, r, p, se = linregress(np.arange(1, yield_val.shape[0] + 1), yield_val.Value)
        
        # copy the information into the dictionary
        slopes[uniq] = [slope, r, p]
    
    # create a dataframe of the dictionary
    slopes_df = pd.DataFrame(slopes).T
    # Change the column names
    slopes_df.columns = ['slope', 'r', 'p']
    # reset the index
    slopes_df = slopes_df.reset_index()
    
    return slopes_df

# Apply function
slopesData = calculate_slopes(att_data)
# save to a file
slopesData.to_csv(r"path_to_dataframe.csv")

In [ ]:
# let's read slopesData
# Read the slope data
slopeData = pd.read_csv('path_to_dataframe.csv')
slopeData.head(10)
slopeData.rename(columns={'Unnamed: 0': 'Id', 'slope': 'Slope'}, inplace = True)
list(slopeData)
slopeData.tail()

### Creating individual trendlines of Yield at each county.

In [ ]:
# Calculating max and minimum values of the yield
max_yield = att_data['Value'].max()
min_yield = att_data['Value'].min()

# Get unique state county combination
stateCountyUnique = np.unique(att_data['State_County'].values)
# stateCountyUnique

# run for loop to get figure in each county
for uniq in stateCountyUnique:
    yield_val = att_data[att_data['State_County'] == uniq].sort_values(by='Year')
    yield_val = yield_val.set_index('Year')
    slope, inter, r, p, se = linregress(np.arange(1, yield_val.shape[0] + 1), yield_val.Value)
    
    # create a figure plot
    fig, ax = plt.subplots(1,1, figsize=(2,2), dpi=72)
    ax.plot(yield_val['Value'], color='blue')
    ax.set_ylabel('Yield(Bu/Acre)')
    ax.set_xlabel('Year')
    ax.set_ylim(bottom=min_yield, top=max_yield)
    ax.text(0.04, 0.96,
            f"{yield_val['County'][0].capitalize()} County, {yield_val['State'][0].capitalize()}",
            ha='left', va='top', transform=ax.transAxes, fontsize=11)
    ax.text(0.04, 0.08, f"Trend Slope: {slope:.2f}\np-value: {p:.3f}",
           ha='left', va='top', transform = ax.transAxes)
    # save the figure
    outName = rf"Plots\{slopeData['index'][0]+'.png'}"
    fig.savefig(outName, dpi=72, bbox_inches='tight')

### The Fun part, Web Mapping.

In [ ]:
# Read the slope data
data = slopeData


# read shapefile data
gpd_data = gpd.read_file(r"NC_USA/NorthCentral.shp")
gpd_data.to_file('myshpfile.geojson', driver='GeoJSON')

shape_geo = gpd.read_file(r"myshpfile.geojson")
# shape_geo.head()

shape_geo=shape_geo[['FID_Georgi', 'NAME_1', 'NAME_2', 'geometry']]
shape_geo.tail(10)

In [ ]:
shape_geo.rename(columns = {'FID_Georgi':'ID', 'NAME_1':'State', 'NAME_2': 'County'}, inplace = True)
shape_geo.tail()

In [ ]:
data.rename(columns = {'Id':'ID'}, inplace = True)
data.tail()

In [ ]:
# # adding new column
# id = [i for i in range(len(shape_geo.axes[0]))]
# shape_geo['Id'] = id

# overlaying our spatial data to slopedata which store information about
# our location data
mergedData = shape_geo.merge(data, on='ID')

# Adding more columns
# County = [county for county in att_data['County'].values]
# States = [states for states in att_data['State'].values]
# mergedData['County'] = County
# mergedData['States'] = States
mergedData.head()

In [ ]:
png = []
directory = r"Plots"
for png_s in os.scandir(directory):
    if png_s.is_file():
        png.append(png_s.name)

    print(png)

mergedData['images'] = png
mergedData.head()

In [ ]:
x_map=shape_geo.centroid.x.mean()
y_map=shape_geo.centroid.y.mean()

# Create a custom scale for the legend
# I'm using the quantile classification to make a custom scale

customScale = (data['Slope'].quantile((0, 0.2, 0.4, 0.6, 0.8, 1))).tolist()

# Initialize the map and store it in a foliumMap object
foliumMap = folium.Map(location=[y_map, x_map], zoom_start=6)

# This is the folium object
ch = folium.Choropleth(
     geo_data = shape_geo, 
     data = mergedData, 
     columns = ['ID', 'Slope'], # specify columns you need to use, we only need these two
     key_on = 'feature.properties.ID', # specify Id column, which is unique
     bins = customScale, # defining the scale
     fill_color='YlOrRd', # Defining the color scale
     nan_fill_color = 'white',
     fill_opacity = 0.7, # some transparency on the fill color
     line_opacity = 0.2, # some fill color on the line color
     legend_name = 'Trend', # legend name
     highlight = True, # highlighting shape when hovering
     line_color = 'black').add_to(foliumMap) # Add this object to the defined map

foliumMap

### Let's add some popup to our map.

In [ ]:
from pathlib import Path
# assign directory
directory = r"Plots"
# iterate over files in
# that directory
# for imageFiles in os.scandir(directory):
#     print(imageFiles.name)

# run a for loop for every raw in the dataframe
for i in range(mergedData.shape[0]):
    image_name = mergedData.loc[i, 'images']
    image_path = os.path.join(r"Plots", image_name)
    encoded = base64.b64encode(open(image_path, 'rb').read())
    html = '<img src="data:image/png;base64,{}">'.format
    width, height = 4, 4 # smaller width and height of each figure
    resolution = 72 # Little lower resolution
    
    # Create the frame, note that we are adding some values(20 and 30).
    # This will help user to avoid scrolling in the popped up figures
    iframe = IFrame(html(encoded.decode("UTF-8")), width=(width * resolution)+20, height=(height * resolution)+30)
    popup = folium.Popup(iframe, maxWidth=2650) # This is the popup object.
    
    # The style function is important. This is a lambda function with style properties.
    # It says the clickable polygon should not have any fill or line colors
    style_function = lambda x: {'fillColor': '#ffffff',
                                'color': '#000000',
                                'fillOpacity': 0.1,
                                'weight': 0.1}
    
    # Create a geojson object of each polygon. Note that, this is practically invisible.
    b = folium.GeoJson(shape_geo.iloc[i, 3], style_function=style_function)
    b.add_child(popup) # Add the popup to it
    ch.add_child(b) # Add the geojson into the map
foliumMap

### Adding a title and saving as htmlfile.

In [ ]:

# Adding map title
text = 'A 10-year Spatiotemporal Trend of Yield in Georgia State'
title_html = '''
                <h3>align="left" style="font-size:22px"><b>{}</b></h3>
             '''.format(text)

foliumMap.get_root().html.add_child(folium.Element(title_html))

# Save the html file
foliumMap.save(r"Result/cornYieldMap.html")